In [7]:
from dotenv import load_dotenv
load_dotenv

import os

from typing_extensions import TypedDict, Annotated
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.prebuilt import tool_node, tools_condition
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command

In [3]:
class State(TypedDict):
    messages : Annotated[list, add_messages]

llm = ChatGroq(
    groq_api_key = os.getenv("GROQ_API_KEY"),
    model = "llama-3.1-8b-instant"
    )

#response = llm.invoke("Confirm you're active!")
#print(response.content)


I'm active and ready to assist you with any questions or topics you'd like to discuss. How can I help you today?


In [ ]:
# Tool 1: Get stock price
@tool
def get_stock_price(symbol: str) -> float:
    ''' 
    Return the current price of a stock symbol.
    :param symbol: Stock symbol (e.g., "AAPL", "GOOGL").
    :return: Current price of the stock as a float.
    '''
    return {
        "AAPL": 175.32,
        "GOOGL": 2850.45,
        "AMZN": 143.67,
        "MSFT": 330.12,
        "TSLA": 255.78,
        "META": 312.56,
        "NFLX": 410.23,
        "NVDA": 480.99,
        "IBM": 135.67,
        "ORCL": 123.45
    }.get(symbol.upper(), 0.0)

# Tool 2: Convert currency
@tool
def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
    ''' 
    Convert an amount from one currency to another.
    :param amount: The amount to convert.
    :param from_currency: The currency to convert from (e.g., "USD").
    :param to_currency: The currency to convert to (e.g., "EUR").
    :return: The converted amount as a float.
    '''
    exchange_rates = {
        ("USD", "EUR"): 0.85,
        ("EUR", "USD"): 1.18,
        ("USD", "GBP"): 0.75,
        ("GBP", "USD"): 1.33,
    }
    return amount * exchange_rates.get((from_currency.upper(), to_currency.upper()), 1.0)

# Tool 3: Get weather information
@tool
def get_weather(city: str) -> str:
    ''' 
    Get the current weather information for a city.
    :param city: The name of the city (e.g., "New York").
    :return: A string describing the weather (e.g., "Sunny, 25°C").
    '''
    weather_data = {
        "New York": "Sunny, 25°C",
        "London": "Cloudy, 18°C",
        "Paris": "Rainy, 20°C",
        "Tokyo": "Clear, 30°C",
    }
    return weather_data.get(city, "Weather data not available")

# Tool 4: Calculate factorial
@tool
def calculate_factorial(n: int) -> int:
    ''' 
    Calculate the factorial of a number.
    :param n: The number to calculate the factorial for.
    :return: The factorial of the number as an integer.
    '''
    if n == 0:
        return 1
    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
    return factorial

# Tool 5: Buy stock
@tool
def buy_stock(stock_name: str, quantity: int, total_price: str) -> str:
    ''' 
    Simulate buying a stock.
    :param stock_name: The name of the stock to buy (e.g., "AAPL").
    :param quantity: The number of stocks to buy.
    :return: A confirmation message as a string.
    '''
    decision : str = interrupt(" Are you sure you want to buy {quantity} of {str} stocks,\n total price : {total_ptice} :  Yes/No? ")
    if decision.lower() == "yes":
        return f"Bought {quantity} shares of {stock_name.upper()} for price : {total_price}."
    else :
        return f"Declined"

# Define the tools list
tools = [get_stock_price, convert_currency, get_weather, calculate_factorial, buy_stock]

In [ ]:
state = StateGraph